In [0]:
import torch
import torch.nn as nn

In [0]:
def topological_sort(g):
  pass

In [0]:
def activate(x, activation):
  pass

In [0]:
class wann:
  def __init__(input_dim, num_classes):
    """
    Initializes a WANN with no connections.
    """
    self.input_dim = input_dim
    self.num_classes = num_classes
    self.n = input_dim + num_classes
    self.g = [set() for i in range(self.n)]
    self.activations = [None] * elf.n
  
  def add_node(u, v, activation):
    """
    Splits the edge u->v to u->x->v and sets activation of node x.
    """
    x = self.n
    
    self.n += 1
    
    self.g.append(set())
    self.g[u].remove(v)
    self.g[u].add(x)
    self.g[-1].add(v)
    
    self.activations.append(activation)
  
  def add_edge(u, v):
    """
    Adds edge u->v.
    """
    self.g[u].add(v)
  
  def change_activation(v, activation):
    """
    Changes activation of node v.
    """
    self.activations[v] = activation
  
  def forward(input, weight):
    """
    Calculates forward pass using shared weight.
    """
    assert len(input) == self.input_dim
    output = input + [0] * (self.n - self.input_dim)
    l = topological_sort(self.g)
    for v in l:
      a = activate(output[v], self.activations[v])
      for w in self.g[v]:
        output[w] += a
    return output[self.input_dim : self.input_dim + self.num_classes]